In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 31, bias= True),
            nn.ReLU(),
            nn.Linear(31, 31, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(31, 30, bias = True),
            nn.ReLU(),
            nn.Linear(30, 30, bias= True),
            nn.ReLU(),
            nn.Linear(30, 30, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(30, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Linear(20, 15, bias= True),
            nn.ReLU(),
            nn.Linear(15, 10, bias= True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=100, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 100)

In [5]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=31, bias=True)
    (1): ReLU()
    (2): Linear(in_features=31, out_features=31, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=31, out_features=30, bias=True)
    (6): ReLU()
    (7): Linear(in_features=30, out_features=30, bias=True)
    (8): ReLU()
    (9): Linear(in_features=30, out_features=30, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.6, inplace=False)
    (12): Linear(in_features=30, out_features=25, bias=True)
    (13): ReLU()
    (14): Linear(in_features=25, out_features=20, bias=True)
    (15): ReLU()
    (16): Linear(in_features=20, out_features=15, bias=True)
    (17): ReLU()
    (18): Linear(in_features=15, out_features=10, bias=True)
    (19): ReLU()
    (20): Linear(in_features=10, out_features=5, bias=True)
    (21): ReLU()
  )
  (classify): Sequential(
    (0): Linear(in_features=5, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay= 0.00001)

In [7]:
for epoch in range(500):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 3338.6132078170776
loss - 3314.9166523218155
loss - 3311.9634160995483
loss - 3310.011078953743
loss - 3307.3223690390587
loss - 3303.5523953437805
loss - 3300.8838123083115
loss - 3300.076217651367
loss - 3298.531846702099
loss - 3297.758933007717
loss - 3296.3846630454063
loss - 3295.898630142212
loss - 3295.540343761444
loss - 3294.730865716934
loss - 3293.7247482538223
loss - 3292.9952949881554
loss - 3292.4407187104225
loss - 3291.575024485588
loss - 3290.695299744606
loss - 3290.5020881295204
loss - 3289.237719833851
loss - 3288.506616473198
loss - 3288.746652305126
loss - 3288.1186366677284
loss - 3287.765330016613
loss - 3287.7912868261337
loss - 3286.9779269099236
loss - 3286.275299370289
loss - 3285.742747604847
loss - 3285.2451055049896
loss - 3285.298554599285
loss - 3284.931565821171
loss - 3283.9746918082237
loss - 3284.5658054351807
loss - 3284.208101928234
loss - 3282.731676697731
loss - 3283.4207869172096
loss - 3283.208724975586
loss - 3282.538392364979
loss - 

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6071441367202894
